<a href="https://colab.research.google.com/github/zhuoyingcai/NYC-Housing-Prices-Analysis/blob/master/NYC_Housing_Prices_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Section

In [0]:
import os

# Data
import numpy as np
import pandas as pd

# Plot
import matplotlib.pyplot as plt


# Path Functions

In [0]:
def ensure_dir(path_to_dir):
  try:
      os.makedirs(path_to_dir)
  except FileExistsError:
      # directory already exists
      pass

# Config

In [3]:
# path config
annualizedSalesUpdate_data_dir = '/content/drive/My Drive/Colab Notebooks/dataScience/annualizedSalesUpdate'
data_dir = '/content/drive/My Drive/Colab Notebooks/dataScience/data'

# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# make sure dir exist
ensure_dir(data_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Load and summarize data

In [0]:
# define dataframe
df_bronx = pd.DataFrame()
df_brooklyn = pd.DataFrame()
df_manhattan = pd.DataFrame()
df_queens = pd.DataFrame()
df_statenisland = pd.DataFrame()

In [6]:
# load bronx data
for year in range(2006,2019):
    path = annualizedSalesUpdate_data_dir + '/%s_bronx.xls' % (year)
    data = pd.read_excel(path, header=4)
    df_bronx = df_bronx.append(data, sort=False)
"""
# load brooklyn data
for year in range(2006,2019):
    path = annualizedSalesUpdate_data_dir + '/%s_brooklyn.xls' % (year)
    data = pd.read_excel(path, header=4)
    df_brooklyn = df_brooklyn.append(data, sort=False)
    
# load manhattan data
for year in range(2006,2019):
    path = annualizedSalesUpdate_data_dir + '/%s_manhattan.xls' % (year)
    data = pd.read_excel(path, header=4)
    df_manhattan = df_manhattan.append(data, sort=False)

# load queens data
for year in range(2006,2019):
    path = annualizedSalesUpdate_data_dir + '/%s_queens.xls' % (year)
    data = pd.read_excel(path, header=4)
    df_queens = df_queens.append(data, sort=False)
    
# load statenisland data
for year in range(2006,2019):
    path = annualizedSalesUpdate_data_dir + '/%s_statenisland.xls' % (year)
    data = pd.read_excel(path, header=4)
    df_statenisland = df_statenisland.append(data, sort=False)
"""

"\n# load brooklyn data\nfor year in range(2006,2019):\n    path = annualizedSalesUpdate_data_dir + '/%s_brooklyn.xls' % (year)\n    data = pd.read_excel(path, header=4)\n    df_brooklyn = df_brooklyn.append(data, sort=False)\n    \n# load manhattan data\nfor year in range(2006,2019):\n    path = annualizedSalesUpdate_data_dir + '/%s_manhattan.xls' % (year)\n    data = pd.read_excel(path, header=4)\n    df_manhattan = df_manhattan.append(data, sort=False)\n\n# load queens data\nfor year in range(2006,2019):\n    path = annualizedSalesUpdate_data_dir + '/%s_queens.xls' % (year)\n    data = pd.read_excel(path, header=4)\n    df_queens = df_queens.append(data, sort=False)\n    \n# load statenisland data\nfor year in range(2006,2019):\n    path = annualizedSalesUpdate_data_dir + '/%s_statenisland.xls' % (year)\n    data = pd.read_excel(path, header=4)\n    df_statenisland = df_statenisland.append(data, sort=False)\n"

In [10]:
# df_bronx.head(20)
df_bronx.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 88640 entries, 0 to 7697
Data columns (total 21 columns):
BOROUGH                           88640 non-null int64
NEIGHBORHOOD                      88640 non-null object
BUILDING CLASS CATEGORY           88640 non-null object
TAX CLASS AT PRESENT              88640 non-null object
BLOCK                             88640 non-null int64
LOT                               88640 non-null int64
EASE-MENT                         80942 non-null object
BUILDING CLASS AT PRESENT         88640 non-null object
ADDRESS                           88640 non-null object
APARTMENT NUMBER                  81666 non-null object
ZIP CODE                          88640 non-null int64
RESIDENTIAL UNITS                 88640 non-null int64
COMMERCIAL UNITS                  88640 non-null int64
TOTAL UNITS                       88640 non-null int64
LAND SQUARE FEET                  88640 non-null int64
GROSS SQUARE FEET                 88640 non-null int64
YEAR 